In [2]:
import os
import tqdm
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf
from tfcf.metrics import mae
from tfcf.metrics import rmse
from tfcf.datasets import ml1m
from tfcf.config import Config
from tfcf.models.svd import SVD
# from tfcf.models.svd import SVDPP
# from sklearn.model_selection import train_test_split

dir_ = '../../data/'
# file_name = 'normalized_filter_track_5_user_100.csv'
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'

In [3]:
# Note that x is a 2D numpy array, 
# x[i, :] contains the user-item pair, and y[i] is the corresponding rating.

df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))

x_train = np.loadtxt(os.path.join(dir_, 'train_x_' + file_name), delimiter=',')
y_train = np.loadtxt(os.path.join(dir_, 'train_y_' + file_name), delimiter=',')

In [5]:
config = Config()
config.num_users = len(df['uid'].unique())
config.num_items = len(df['tid'].unique())
config.min_value = df['rating'].min()
config.max_value = df['rating'].max()

In [6]:
# For top N
pd_train = pd.read_pickle(os.path.join(dir_, 'train_' + file_name[:-3] + 'pkl'))
pd_all = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
num_users = len(pd_all['uid'].unique())
num_tracks = len(pd_all['tid'].unique())
print(num_users,num_tracks)

120 26438


In [7]:
track_ids = []
for i in range(num_tracks):
    track_ids.append(i)

all_tracks = pd.DataFrame()
all_tracks['tid'] = track_ids
all_tracks['count'] = 0

In [8]:
x_test = []

for i in tqdm(range(num_users)):
    
    user = pd_train[pd_train['uid']==i]
    top_n = all_tracks.set_index('tid').add(user.set_index('tid'), fill_value=0).reset_index()
    top_n = top_n[top_n['count']==0]
    top_n['uid'] = i
    top_n = top_n[['uid', 'tid']]
    top_n = top_n.values.tolist()
    x_test.extend(top_n)

In [9]:
x_test = np.array(x_test)
print(x_test[0].shape)

(2,)


In [10]:
with tf.Session() as sess:
    # For SVD++ algorithm, if `dual` is True, then the dual term of items' 
    # implicit feedback will be added into the original SVD++ algorithm.
    # model = SVDPP(config, sess, dual=False)
    # model = SVDPP(config, sess, dual=True)
    model = SVD(config, sess)
#     model.train(x_train, y_train, validation_data=(
#         x_test, y_test), epochs=5, batch_size=1024)

    model.train(x_train, y_train, epochs=5, batch_size=1024)
        
    y_pred = model.predict(x_test)
#     print('rmse: {}, mae: {}'.format(rmse(y_test, y_pred), mae(y_test, y_pred)))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
137/137 [==============================] - 1s 8ms/step - rmse: 1.7310 - mae: 1.5693
Epoch 2/5
137/137 [==============================] - 1s 7ms/step - rmse: 1.7065 - mae: 1.5458
Epoch 3/5
137/137 [==============================] - 1s 7ms/step - rmse: 1.6603 - mae: 1.5007
Epoch 4/5
137/137 [==============================] - 1s 7ms/step - rmse: 1.5840 - mae: 1.4257
Epoch 5/5
137/137 [==============================] - 1s 7ms/step - rmse: 1.4838 - mae: 1.3258


In [11]:
print(y_pred)

[1.7156279 1.8180833 2.199155  ... 1.5954353 2.142235  2.0473254]


In [12]:
df = pd.DataFrame(y_pred, columns=['rating'])
df2 = pd.DataFrame(x_test, columns=['uid', 'tid'])
df2.insert(2, 'rating', y_pred, False) 
df2

,uid,tid,rating
0,0,0,1.715628
1,0,5,1.818083
2,0,7,2.199155
3,0,8,2.362023
4,0,9,2.480099
...,...,...,...
3032324,119,26432,2.205485
3032325,119,26433,1.915779
3032326,119,26434,1.595435
3032327,119,26436,2.142235


In [13]:
df2.to_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))